In [21]:
import os
from glob import glob

import numpy as np

from pyuvdata import UVData

In [61]:
SAVEPATH = '/lustre/aoc/projects/hera/afortino/2458111'
dfiles = sorted(glob('/lustre/aoc/projects/hera/afortino/2458111/zen.2458111.?????.??.HH.uvOCRS'))

In [71]:
if len([dfile for dfile in dfiles if 'xx' in dfile]) == \
len([dfile for dfile in dfiles if 'yy' in dfile]) == \
len([dfile for dfile in dfiles if 'xy' in dfile]) == \
len([dfile for dfile in dfiles if 'yx' in dfile]):
    print 'xx, yy, xy, and yx files are present in equal numbers'
else: raise Exception('xx, yy, xy, and yx files are NOT present in equal numbers')

xx, yy, xy, and yx files are present in equal numbers


In [74]:
numfiles_uniq = len(dfiles) / 4
print numfiles_uniq

for batch_4pol in np.array_split(dfiles, numfiles_uniq):
    batch_4pol = batch_4pol.tolist()
    uvd = UVData()
    uvd.read_miriad(
        batch_4pol,
        run_check=False,
        check_extra=False,
        run_check_acceptability=False,
        ant_str='cross')
    
    JDT = np.round(np.unique(uvd.time_array)[0], 5)
    JDT = '{:.5f}'.format(JDT)
    JD = JDT.split('.')[0]
    JDdec = JDT.split('.')[1]
    pols = uvd.polarization_array.join('_')
    
    hdf5_file = 'zen.{JD}.{JDdec}.{pols}.HH.uvOCRS.hdf5'.format(
        JD=JD,
        JDdec=JDdec,
        pols=pols)
    print hdf5_file
    break
    uvd.write_uvh5(hdf5_file, clobber=True)
    del uvd

73


AttributeError: 'numpy.ndarray' object has no attribute 'join'

In [55]:
testhdf5 = os.path.join(SAVEPATH, 'test.hdf5')
uvd.write_uvh5(testhdf5, clobber=True)

File exists; clobbering


In [64]:
uvd.read_miriad??

In [ ]:
%%time
for dfile in dfiles:
    print dfile
    print 'Loading UVdata object...'
    uvd = UVData()
    print 'Loading data file...'
    uvd.read_miriad(
        dfile,
        run_check=False,
        check_extra=False,
        run_check_acceptability=False,
        ant_str='cross')
    print 'Writing to hdf5...'
    uvd.write_uvh5(dfile + '.hdf5', clobber=True)    
    del uvd

In [ ]:
batch_mid = np.array(batch[1:-1])
numbatches = len(batch) / 8

uvd = UVData()
uvd.read_miriad(batch[0])
print batch[0]

for batch_small in np.array_split(batch_mid, numbatches):
    uvd_batch_small = UVData()
    uvd_batch_small.read_miriad(batch_small[0])
    print batch_small[0]
    
    for dfile in batch_small[1:]:
        uvdi = UVData()
        uvdi.read_miriad(dfile)
        print dfile

        uvd_batch_small += uvdi
        del uvdi
    uvd += uvd_batch_small
    del uvd_batch_small
else:
    uvdf = UVData()
    uvdf.read_miriad(batch[-1])
    print batch[-1]
    uvd += uvdf

In [ ]:
batch_jd_start = np.round(np.unique(uvd.time_array)[0], 5)
batch_jd_stop = np.round(np.unique(uvdf.time_array)[0], 5)
JD0 = '{:.5f}'.format(batch_jd_start)
JDf = '{:.5f}'.format(batch_jd_stop)
JD = JD0.split('.')[0]
JD0 = JD0.split('.')[1]
JDf = JDf.split('.')[1]
numfiles = str(len(batch))
pols = '_'.join([pol.lower() for pol in uvd.get_pols()])

dirname = 'zen.{JD}.{JD0}_{numfiles}_{JDf}.{pols}.HH.{ext}'.format(
    JD=JD,
    JD0=JD0,
    JDf=JDf,
    numfiles=numfiles,
    pols=pols,
    ext=ext)
dirname = os.path.join(SAVEPATH, dirname)
print dirname

In [ ]:
os.system('mkdir -p {dirname}'.format(dirname=dirname))

In [ ]:
uvd.batch_jd_start = np.unique(uvd.time_array)[0]
uvd.batch_jd_stop = np.unique(uvdf.time_array)[0]
uvd.batch_numfiles = len(batch)
uvd.batch_pols = uvd.get_pols()
uvd.batch_dirname = dirname
uvd.batch_ext = ext
uvd.batch_hdf5name = 'UVD.hdf5'

In [ ]:
hdf5name = os.path.join(dirname, 'UVD.hdf5')
print hdf5name

In [ ]:
uvd.write_uvh5(hdf5name)